In [2]:
import os
from dotenv import load_dotenv

load_dotenv(verbose=True)
OPENAI_API_KEY = os.getenv("OPEN_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [4]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Automated testing: Langsmith can be used to write and execute automated tests for your software application. This can help in quickly identifying bugs and issues in the code.\n\n2. Performance testing: Langsmith can be used to simulate various user scenarios and test the performance of your application under different conditions. This can help in identifying bottlenecks and improving the overall performance of the application.\n\n3. Integration testing: Langsmith can be used to test the integration of different components of your application. This can help in ensuring that all the components work together seamlessly.\n\n4. Regression testing: Langsmith can be used to run regression tests to ensure that new code changes do not introduce any new bugs or issues in the application.\n\n5. Load testing: Langsmith can be used to simulate a high load on the application and test its performance under heavy traffic. This c

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [6]:
chain = prompt | llm

In [7]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content="Langsmith is a powerful tool that can greatly assist with testing in various ways. Here are some ways in which Langsmith can help with testing:\n\n1. **Automated Testing**: Langsmith can be used to automate the testing of various components of a system, such as APIs, web services, and databases. By writing scripts in Langsmith, testers can automate the process of sending requests, validating responses, and performing various checks to ensure the functionality of the system.\n\n2. **Data Generation**: Langsmith can generate realistic test data that can be used to simulate different scenarios during testing. This can help in testing the system's robustness and performance under various conditions.\n\n3. **Integration Testing**: Langsmith can be used to perform integration testing by simulating interactions between different components of a system. By scripting these interactions in Langsmith, testers can ensure that all components work together seamlessly.\n\n4. **Perf

In [8]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [9]:
chain = prompt | llm | output_parser

In [10]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can help with testing in multiple ways:\n\n1. Automated Test Generation: Langsmith can automatically generate unit tests, integration tests, and end-to-end tests for your codebase. This can help in increasing test coverage and finding bugs early in the development process.\n\n2. Test Data Generation: Langsmith can generate test data for your application, including edge cases and boundary values. This can help in validating the robustness and reliability of your code.\n\n3. Test Scenario Generation: Langsmith can generate test scenarios based on different user interactions and workflows. This can help in testing the functionality and usability of your application comprehensively.\n\n4. Test Coverage Analysis: Langsmith can analyze the test coverage of your codebase and identify areas that require more testing. This can help in ensuring that your tests are effectively covering all parts of your application.\n\n5. Continuous Integration: Langsmith can be integrated into your CI

In [11]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [12]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [14]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [15]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [16]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing you to visualize test results.'

In [17]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [18]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by providing features such as prototyping, debugging, initial test set creation, comparison view, playground environment for rapid iteration and experimentation, beta testing, collecting feedback, annotating traces, adding runs to datasets, production monitoring, and A/B testing. These features allow developers to test and evaluate the performance of their LLM applications at various stages of development, helping them identify and resolve issues, track improvements, and gather feedback from users.


In [19]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [20]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="This allows you to quickly test out different prompts and models. You can open the playground from any prompt or model run in your trace.\nEvery playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing\u200bBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it’s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it’s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Collecting Feedback\u200bWhen launching your application 

In [21]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [22]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understan

In [23]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)